In [2]:
#tensorflow_version 2.x 
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf     
import pandas as pd


CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLenght', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
#Constants defined to help us

train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
#We use "keras"(a module inside tensorflow) to grab our datasets and read them into a pandas dataframe

print(train.head())
train_y = train.pop('Species')
test_y = test.pop('Species')
print(train.head())

print(train.shape)

#Input Function
def input_fn(features, labels, training=True, batch_size=256):
    #Convert inputs to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices(dict(features),labels)
    
    #shuffle and repeat if you are in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

# Feature columsn describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

#Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    #Two Hidden Layers of 30 and 10 nodes respectively.
    hidden_units = [30, 10],
    #The model must choose between 3 classes
    n_classes=3)

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps = 5000
)
#We include a lambda to avoid creating an inner function previously

   SepalLength  SepalWidth  PetalLenght  PetalWidth  Species
0          6.4         2.8          5.6         2.2        2
1          5.0         2.3          3.3         1.0        1
2          4.9         2.5          4.5         1.7        2
3          4.9         3.1          1.5         0.1        0
4          5.7         3.8          1.7         0.3        0
   SepalLength  SepalWidth  PetalLenght  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3
(120, 4)
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLenght', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), N

AttributeError: module 'tensorflow' has no attribute 'estimator'

In [ ]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest Set Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [ ]:
def input_fn(features, batch_size=256):
    #Convert the inputs to a Dataset without labels
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict={}

print("Please type numeric values as prompted.")
for feature in features:
    valid = True
    while valid:
        val = input(feature+": ")
        if not val.isdigit(): valid = False
        
    predict[feature] = [float(val)]
    
predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100*probability))